# Analysis and Visualization

Let's create a visualization of a single meme:

## 0. Setup

In [1]:
import os
import os.path

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [2]:
# Parameters

# Folders on local machine where to create the output and temporary files:
input_path = "wikidata"
output_path = "projects"
project_name = "tutorial-kypher"

In [3]:
big_files=["label"]

additional_files = {
    "P31": "derived.P31.tsv.gz",
    "items": "claims.wikibase-item.tsv.gz",
    "P1963": "derived.P1963computed.count.star.tsv.gz",
    "external": "claims.external-id.tsv.gz",
    "indegree": "metadata.in_degree.tsv.gz",
    "outdegree": "metadata.out_degree.tsv.gz",
    "pagerank": "metadata.pagerank.directed.tsv.gz"
}

ck = ConfigureKGTK(big_files)
ck.configure_kgtk(input_graph_path=input_path, 
                  output_path=output_path, 
                  project_name=project_name,
                  additional_files=additional_files)

User home: /Users/filipilievski
Current dir: /Users/filipilievski/mcs/imkg
KGTK dir: /Users/filipilievski/mcs
Use-cases dir: /Users/filipilievski/mcs/use-cases


In [4]:
ck.print_env_variables()

KGTK_OPTION_DEBUG: false
GRAPH: wikidata
KGTK_LABEL_FILE: wikidata/labels.en.tsv.gz
kypher: kgtk query --graph-cache projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
KGTK_GRAPH_CACHE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
OUT: projects/tutorial-kypher
EXAMPLES_DIR: /Users/filipilievski/mcs/examples
TEMP: projects/tutorial-kypher/temp.tutorial-kypher
STORE: projects/tutorial-kypher/temp.tutorial-kypher/wikidata.sqlite3.db
kgtk: kgtk
USE_CASES_DIR: /Users/filipilievski/mcs/use-cases
label: wikidata/labels.en.tsv.gz
P31: wikidata/derived.P31.tsv.gz
items: wikidata/claims.wikibase-item.tsv.gz
P1963: wikidata/derived.P1963computed.count.star.tsv.gz
external: wikidata/claims.external-id.tsv.gz
indegree: wikidata/metadata.in_degree.tsv.gz
outdegree: wikidata/metadata.out_degree.tsv.gz
pagerank: wikidata/metadata.pagerank.directed.tsv.gz


In [134]:
!kgtk query -i $TEMP/templates.kgtk.gz \
            --match '(:`kym:distracted-boyfriend`)-[r]->()' \
            --where 'r.label in ["kym:parent", "kym:year", "rdf:type", "m4s:fromAbout", "m4s:fromTags"]' \
            -o $TEMP/db_subject.kgtk.gz

Create node file:

In [135]:
!kgtk query -i $label -i $TEMP/templates.kgtk.gz \
            --match 'templates: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as node1, r.label as label, l as node2' / deduplicate \
            -o $TEMP/labelfile.kgtk.gz 

In [136]:
!kgtk query -i $TEMP/labelfile.kgtk.gz -i $TEMP/db_subject.kgtk.gz \
            --match 'db: ()-[]->(n), \
                label: (n)-[r]->(l)' \
            --return 'n as id, l as label' / deduplicate \
            -o $TEMP/nodefile.kgtk.gz 

In [137]:
kgtk('''
    cat -i $TEMP/nodefile.kgtk.gz
''')

,id,label
0,Q1634416,'stock photography'@en
1,Q225613,'iStockphoto'@en
2,Q2920963,'stew'@en
3,Q6002242,'image macro'@en
4,Q622404,'boyfriend'@en


In [138]:
kgtk('''
    cat -i $TEMP/db_subject.kgtk.gz
''')

,node1,label,node2
0,kym:distracted-boyfriend,kym:parent,kym:object-labeling
1,kym:distracted-boyfriend,kym:year,2017
2,kym:distracted-boyfriend,m4s:fromAbout,Q1634416
3,kym:distracted-boyfriend,m4s:fromAbout,Q622404
4,kym:distracted-boyfriend,m4s:fromTags,Q1634416
5,kym:distracted-boyfriend,m4s:fromTags,Q225613
6,kym:distracted-boyfriend,m4s:fromTags,Q2920963
7,kym:distracted-boyfriend,m4s:fromTags,Q6002242
8,kym:distracted-boyfriend,rdf:type,kym:Meme
9,kym:distracted-boyfriend,rdf:type,kymt:image-macro


In [141]:
kgtk("""visualize-graph 
        -i $TEMP/db_subject.kgtk.gz
        --node-file $TEMP/nodefile.kgtk.gz
        --show-text above
        --tooltip-column label
        --direction arrow
        --edge-color-column label
        --edge-color-style d3.schemeDark2
        -o distracted.graph.html""")

In [128]:
!grep 'mordor' imkg02/templates.kym.nt

<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q219504> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q4465> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q2338287> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q132311> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q202886> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wiki/Q15228> .
<https://knowyourmeme.com/memes/one-does-not-simply-walk-into-mordor> <https://meme4.science/fromTags> <https://www.wikidata.org/wik

Issue: graph seems to be missing crucial information!

Next step: improve the node file to include colors for relations